In [1]:
import numpy as np
import graphtools as gt
from importlib import reload
import pandas as pd
import matplotlib as plt

%matplotlib notebook

In [5]:
# Hyperparams
vfname = "veldata/secondring"
info = {}
infofname = vfname+".info"
print("Creating info dict")
info = gt.get_info_dict(infofname)
for key,val in sorted(info.items()):
    print(key,"\t",val)

Creating info dict
nTG 	 144
n_drivers 	 189515
n_points 	 370404
source 	 /home/michael/msc/summer17/traffic/data/OUT0_FiveRing150buffer
tcutoff 	 1.0
tglen 	 10
velmin 	 0.0
xmax 	 9917.53639401
xmin 	 9907.53639399
ymax 	 4443.05976788
ymin 	 4431.05976788


In [6]:
# Gather nodes
xmin,xmax = vinfo["xmin"]/gt.long2km, info["xmax"]/gt.long2km
ymin,ymax = vinfo["ymin"]/gt.lat2km, info["ymax"]/gt.lat2km
region_gsi = [xmin,xmax,ymin,ymax]
nodes = gt.generate_nodes(region=region_gsi, mindist=0.5, maxdist=2., maxnbr=1e7)
print("Number of nodes", len(nodes.index))
nodes[:3]

Number of nodes 135


,coords,nbrs,coords_km
137,"[116.3672434, 39.9226143]","[397, 353, 703, 704, 705, 320, 317, 872, 321, ...","[9910.635713372798, 4438.495387481918]"
142,"[116.3903093, 39.9269964]","[774, 355, 1074, 1075, 1073, 1070, 143, 1071, ...","[9912.600164240774, 4438.982578287894]"
169,"[116.4447293, 39.8748632]","[376, 170, 1042, 1040, 1041, 388, 1038, 1039, ...","[9917.234946158464, 4433.186540834137]"


In [ ]:
# Create snap files
# Each snap file should have:
# - array of node features
# - array of edge features
# - array of senders
# - array of receivers
#
# the dataframe is a great setup for this

In [ ]:
vdf = gt.get_veldf(vfname,days=[], nTG=vinfo["nTG"], nvel=1e4)
print(len(vdf.index))

In [ ]:
# We have the vdf function
# Need a 